## Homework5-0314
   **截止日期：**<font color=ff0000>**3.17(周四)晚** </font>
   <br><br>**注：**本次作业中完成对应题目的选做题可以不用完成原题

### 1 正则表达式(2分)：
具体格式要求如下：
<br>账号：由英文字母组成，不限大小写。
<br>密码：由英文字母与数字组成且以英文字母开头，最少八位。
<br>电话：11位数字，且以1开头。
<br>生日：年-月-日(YYYY-MM-DD)。

In [11]:
import re
account=""
password=""
telephone=""
birthday=""
#TODO
account = "[A-Za-z]+"
password = "[A-Za-z]\w{7,}"
telephone = "1[0-9]{10}"
birthday = "\d{4}-[0-1][0-9]-[0-3][0-9]"

assert re.match(account,"PyThoN")
assert re.match(password,"Python2022")
assert re.match(telephone,"12345678910")
assert re.match(birthday,"2022-03-14")

<re.Match object; span=(0, 10), match='2022-03-14'>


### 1.5(选做)邮箱的正则表达式匹配（2+0.5分）
邮箱中间须有@，@之前由英文字母、数字、下划线(_)、短横(-)组成，之后由英文字母、数字、下划线(_)、短横(-)、英文句号(.)组成，且必须有至少一个英文句号(.)。（0.5分）

In [36]:
email = "[\w_-]+@[\w_-]+(\.[\w_-]+)+"
assert re.match(email,"2-102@pku.edu.cn")

### 2 yield初步（2分)：
使用yield实现有以下隐含功能的迭代器：默认情况返回给定字符串中的下一个单词；如果send一个字符串，则将其改为全小写并返回。

In [51]:
s="Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0."
#TODO
def gen(st):
    recv = None
    num = 0
    wordlist = st.split(' ')
    while num < len(wordlist):
        if recv != None:
            recv = yield recv.lower()
        else:
            recv = yield wordlist[num]
            num += 1
    
f = gen(s)

#以下为测试
#print(next(f))
#print(next(f))
#print(f.send('ASPaKD'))
#print(next(f))
#for i in f:
#    print(i)

### 3 async/await初步（2分）
采用异步非并发与并发的方式完成代码，并比较完成时间。

### 3.5(选做) async/await初步(2+0.5分)
(选做)(+0.5)如果worker_1只能完成耗时为1或3的任务，worker_2只能完成耗时为5的任务，请完成以下功能：worker发现自己无法完成当前任务，就把该任务交给另一个worker完成（只需修改task_list即可），代码可随意修改。

In [3]:
import asyncio,time
async def working(time):
    await asyncio.sleep(time)
    
towork1 = []
towork2 = []

async def worker_1():
    task_list=[5,1,3]
    for task in task_list:
        if task != 1 and task != 3:
            towork2.append(task)
        else:
            await working(task)
        while len(towork1) != 0:
            task_list.append(towork1[0])
            towork1.remove(towork1[0])
    
async def worker_2():
    task_list=[3,1,5]
    for task in task_list:
        if task != 5:
            towork1.append(task)
        else:
            await working(task)
        while len(towork2) != 0:
            task_list.append(towork2[0])
            towork2.remove(towork2[0])

async def main():
    print(f"started at {time.strftime('%X')}")
    #TODO
    await worker_1()
    await worker_2()
    print(f"finished at {time.strftime('%X')}")
    print("以上为异步非并发情况")
    #以上为异步非并发
    #以下为并发
    towork1.clear()
    towork2.clear()
    print(f"started at {time.strftime('%X')}")
    work1 = asyncio.create_task(worker_1())
    work2 = asyncio.create_task(worker_2())
    await work1
    await work2
    print(f"finished at {time.strftime('%X')}")
    print("以上为并发情况")

await main()

started at 10:49:08
finished at 10:49:22
以上为异步非并发情况
started at 10:49:22
finished at 10:49:32
以上为并发情况


### 4 协程实现轮转调度（2分）
在课件上的轮转调度框架的基础上，完成题目：
<br>worker1与worker2分别有三种不同等级（1,2,3）的任务，task中储存了这几种任务对应的数目（例如worker1有3个等级为1的任务），worker1与worker2一次只可完成一个任务。
<br>请实现：Handle可指定一种等级(例如等级2)的任务，worker1与worker2只需完成该等级的所有任务。
    
### 4.5(选做) 协程实现轮转调度（2+0.5分）：
worker1与worker2按照1-2-3的等级依次完成任务，如果当前等级任务数目为0，则输出“Worker1/2 takes a break.”(认为其完成了该等级任务，不影响循环进行)，如果worker1或worker2已完成所有任务，则由剩下的一个worker独自按照1-2-3的顺序完成任务。例如：worker1完成1级任务->worker2完成2级任务->worker1完成3级任务->worker2完成1级任务……（代码可随意修改！）


In [36]:
waiting_list=[]
worknum = 0

def worker1():
    task={1:3,2:1,3:3}
    tasknum = sum(task.values())
    while True:
        if tasknum == 0:
            break
        rank=yield
        if task[rank] == 0:
            print("worker1 takes a break")
            yield
        if task[rank]>0:
            task[rank]=task[rank]-1
            tasknum -= 1
            print(f'worker1:{task[rank]} remains.task number', rank)
            if task[rank] == 0:
                print("worker1 finishes task number", rank)
            yield
    print('worker1 finished!')
    
def worker2():
    task={1:2,2:3,3:3}
    tasknum = sum(task.values())
    while True:
        if tasknum == 0:
            break
        rank=yield
        if task[rank] == 0:
            print("worker2 takes a break")
            yield
        if task[rank]>0:
            task[rank]=task[rank]-1
            tasknum -= 1
            print(f'worker2:{task[rank]} remains.task number', rank)
            if task[rank] == 0:
                print("worker2 finishes task number", rank)
            yield
    print('worker2 finished!')
    
    
class Handle(object):
    
    def __init__(self,gen):
        self.gen=gen

    def call(self):
        #TODO
        global worknum
        next(self.gen)
        self.gen.send(worknum%3+1)
        worknum += 1
        
    
def RoundRobin(*worker):
    waiting_list.extend(Handle(c) for c in worker)
    #TODO
    while waiting_list:
        for workerwithhandle in waiting_list:
            try:
                workerwithhandle.call()
            except StopIteration:
                waiting_list.remove(workerwithhandle)
    print('All finished!')
    
w1=worker1()
w2=worker2()

RoundRobin(w1,w2)

worker1:2 remains.task number 1
worker2:2 remains.task number 2
worker1:2 remains.task number 3
worker2:1 remains.task number 1
worker1:0 remains.task number 2
worker1 finishes task number 2
worker2:2 remains.task number 3
worker1:1 remains.task number 1
worker2:1 remains.task number 2
worker1:1 remains.task number 3
worker2:0 remains.task number 1
worker2 finishes task number 1
worker1 takes a break
worker2:1 remains.task number 3
worker1:0 remains.task number 1
worker1 finishes task number 1
worker2:0 remains.task number 2
worker2 finishes task number 2
worker1:0 remains.task number 3
worker1 finishes task number 3
worker2 takes a break
worker1 finished!
worker2 takes a break
worker2:0 remains.task number 3
worker2 finishes task number 3
worker2 finished!
All finished!


### 5 生产者/消费者（2分）
采用生产者消费者模型完成给定数据集的检索。这里生产者将检索字符串放入队列中，消费者读取并查找包含字符串的文章，返回给生产者。
<br>需查找：包含（1）全国XX委员会（2）中国XX协会（3）中美建交

In [26]:
news=[]
with open('news.txt','r',encoding='utf-8') as f:
    news=f.readlines()

In [29]:
#查找的文章以news中的文章序号呈现
import nest_asyncio
import asyncio,time
import re
nest_asyncio.apply()

async def consumer(q):
    print('consumer starts.')
    #TODO
    while True:
        searchst = await q.get()
        if searchst is None:
            q.task_done()
            break
        else:
            l = []
            for newspiece in range(len(news)):
                if re.search(searchst, news[newspiece]) != None:
                    l.append(newspiece)
            print(l)
            q.task_done()
    print("consumer ends.")

async def producer(q):
    print('producer starts.')
    #TODO
    searchlist=['全国.*委员会', '中国.*协会', '中美建交']
    for st in searchlist:
        await q.put(st)
    await q.put(None)
    await q.join()
    print("producer ends.")
    
q=asyncio.Queue(maxsize=10)
t0=time.time()
loop=asyncio.get_event_loop()
tasks=[producer(q),consumer(q)]
loop.run_until_complete(asyncio.wait(tasks))
#loop.close()
print(time.time()-t0,"s")

consumer starts.
producer starts.
[0, 8, 39, 50, 74, 86, 90, 92]
[57, 63, 92]
[1, 3, 27, 41, 42, 63, 74, 86, 90, 92]
consumer ends.
producer ends.
0.0020003318786621094 s


### 5.5(选做) 生产者/消费者（2+0.5分）
完成本地C/S架构的生产者消费者模型,并完成以上任务。

In [1]:
%%writefile server.py
#TODO
#服务端，jupyter上不宜运行，故将代码保存到本地，再通过命令行运行
import socket
import time
import re

news=[]
with open('news.txt','r',encoding='utf-8') as f:
    news=f.readlines()

print("initializing server host information")
port = 2828
host = '0.0.0.0'
address = (host, port)
print("initializing server host socket")
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(address)
server.listen(6)
print("waiting for connecting")
while True:
    conn, addr = server.accept()
    print("client information:", addr)
    matchst = conn.recv(100).decode('utf-8', errors='ignore')
    while matchst != '':
        l = []
        for newspiece in range(len(news)):
            if re.search(matchst, news[newspiece]) != None:
                l.append(newspiece)
        conn.send(str(l).encode('utf-8'))
        matchst = conn.recv(100).decode('utf-8', errors='ignore')
    conn.close()

Overwriting server.py


In [1]:
%%writefile client.py
#客户端，jupyter上不宜运行，故将代码保存到本地，再通过命令行运行
import socket
import time
host = '127.0.0.1'
port = 2828
client = socket.socket()
client.connect((host, port))

searchlist=['全国.*委员会', '中国.*协会', '中美建交']
for st in searchlist:
    client.send(st.encode('utf-8'))
    data = client.recv(100).decode('utf-8', errors='ignore')
    print(data)
client.send(''.encode('utf-8'))
client.close()

Overwriting client.py
